In [1]:
from diffractio import degrees, mm, plt, sp, um, np
from diffractio.scalar_masks_XY import Scalar_mask_XY
from diffractio.scalar_masks_XYZ import Scalar_mask_XYZ
from diffractio.scalar_sources_XY import Scalar_source_XY

from scipy.constants import c, epsilon_0, e, m_e, h
import numpy as np
import sympy as sp

from sympy.core.numbers import pi
from IPython.display import display
import numba
from numbalsoda import dop853, lsoda, lsoda_sig
from numba import njit, cfunc
from numpy.lib.scimath import sqrt as csqrt
from numpy.fft import fft2, ifft2, fftshift

In [3]:
numba.jit(nopython = True, cache=True)
def PWD_kernel(Ek, n, k0, k_perp2, dz):
    """
    Step for scalar(TE) Plane wave decomposition(PWD) algorithm.

    Parameters:
        u(np.array): field
        n(np.array): refraction index
        k0(float): wavenumber
        k_perp(np.array): transversal k
        dz(float): increment in distances

    Returns:
        (numpy.array): Field at at distance dz from the incident field

    References:
        1. Schmidt, S. et al. Wave - optical modeling beyond the thin - element - approximation. Opt. Express 24, 30188 (2016).

    """
    H = np.exp(1j * dz * csqrt(n**2 * k0**2 - k_perp2))

    result = (ifft2(fftshift(H * Ek)))
    return result

numba.jit(nopython = True, inline= True, cache=True)
def WPM_schmidt_kernel(u, n, k0, k_perp2, dz):
    """
    Kernel for fast propagation of WPM method

    Parameters:
        u (np.array): fields
        n (np.array): refraction index
        k0 (float): wavenumber
        k_perp2 (np.array): transversal k**2
        dz (float): increment in distances

    References:

        1. M. W. Fertig and K.-H. Brenner, “Vector wave propagation method,” J. Opt. Soc. Am. A, vol. 27, no. 4, p. 709, 2010.

        2. S. Schmidt et al., “Wave-optical modeling beyond the thin-element-approximation,” Opt. Express, vol. 24, no. 26, p. 30188, 2016.
    """
    refraction_indexes = np.unique(n)

    u_final = np.zeros_like(u, dtype=complex)
    Ek = fftshift(fft2(u))
    for m, n_m in enumerate(refraction_indexes):
        u_temp = PWD_kernel(Ek, n_m, k0, k_perp2, dz)
        Imz = (n == n_m)
        u_final = u_final + Imz * u_temp

    return u_final

numba.jit(nopython = True, inline= True, cache=True)
def WPM(x0, y0, z0, t0, wavelength):
    k0 = 2 * np.pi / wavelength

    x = x0
    y = y0
    z = z0
    
    t0.u[:, :, 0] = t0.u0.u
    
    num_x = len(x)
    num_y = len(y)
    
    dx = x[1] - x[0]
    kx = 2 * np.pi / (num_x * dx) * (range(-int(num_x / 2), int(
        num_x / 2)))
    
    dy = y[1] - y[0]
    ky = 2 * np.pi / (num_y * dy) * (range(-int(num_y / 2), int(
        num_y / 2)))

    KX, KY = np.meshgrid(kx, ky)

    k_perp2 = KX**2 + KY**2

    filter_edge = 1
    dz = z[1] - z[0]
    
    num_steps = len(t0.z)
    for j in range(1, num_steps):

        t0.u[:, :, j] = t0.u[:, :, j] + WPM_schmidt_kernel(
            t0.u[:, :, j - 1], t0.n[:, :, j - 1], k0, k_perp2,
            dz) * filter_edge
        
        print(j)

numba.jit(nopython = True, inline= True, cache=True)
def WPM_inplace(x0, y0, z0, t0, wavelength):
    k0 = 2 * np.pi / wavelength

    x = x0
    y = y0
    z = z0

    num_x = len(x)
    num_y = len(y)
    
    dx = x[1] - x[0]
    kx = 2 * np.pi / (num_x * dx) * (range(-int(num_x / 2), int(
        num_x / 2)))
    
    dy = y[1] - y[0]
    ky = 2 * np.pi / (num_y * dy) * (range(-int(num_y / 2), int(
        num_y / 2)))

    KX, KY = np.meshgrid(kx, ky)

    k_perp2 = KX**2 + KY**2

    filter_edge = 1
    dz = z[1] - z[0]
    
    num_steps = len(t0.z)
    for j in range(1, num_steps):
        u0 = t0.u
        t0.u +=  WPM_schmidt_kernel(
            u0, t0.n[:, :, j - 1], k0, k_perp2,
            dz) * filter_edge
        
        print(j)

In [4]:
X, Y, Z = sp.symbols('X Y Z')

#Use scipy's constants
e = e #-1.60217662e-19 #unit C
m = m_e #9.10938356e-31 #unit kg
h_planck = h #6.62607015e-34 #unit J . s 
c_l = c #2.99792458e8 #unit m/s
eps_0_val = epsilon_0 #8.85418782e-12 #permittivity unit F/m
#eps = abs(e)/(2*m*c_l**2) #eps as defined in Hawkes, different from the permittivity of free space. 

#Set initial accelerating voltage. I know it's unrealistic but it gives an output that is easy to plot for now.
phi_0 = 60e5

#Calculate corresponding wavelength of the electron, and divide by 1e-6 so that everything is scaled around the micron unit (this is for the diffractio package)
wavelength = h/(2*abs(e)*m*phi_0)**(1/2)/1e-6

#Set voltage of biprism
V = 200

#Set radius of biprism wire
r = 0.125e-3/1e-6

#Set radius of the condenser surrounding the wire
R = 1e-3/1e-6

phi_biprism = V*(sp.log((X*X+(Z)**2)/(R**2)))/(2*sp.log(r/R))

#Here we write the "refractive index" that the electron would see, using the equation for the electrostatic potential. See Eq 5.17 in Hawkes Principles of Electron Optics
n_eq1 = (1+phi_biprism/phi_0)**(0.5)
n_eq1_lambda = sp.lambdify([Z, X, Y], n_eq1)

#Set up the scale in x and z
x0 = np.linspace(-0.4 * um, 0.4 * um, 256 * 1)
y0 = np.linspace(-0.4 * um, 0.4 * um, 256 * 1)
z0 = np.linspace(-2 * um, 3 * um, 1024 * 1)
xx0, yy0, zz0 = np.meshgrid(x0, y0, z0)

#Create a source plane wave
u0 = Scalar_source_XY(x=x0, y=y0, wavelength=wavelength)
u0.plane_wave(A=1, z0=0 * um, theta=0. * degrees)

t0 = Scalar_mask_XYZ(x=x0, y=y0, z=z0, wavelength=wavelength, n_background=0)
t0.incident_field(u0)

#Create the biprism volume
t0.n = np.complex128(n_eq1_lambda(zz0, xx0, yy0))
t0.cylinder(r0=(0, 0, 0), radius=(r * um, r * um), length = r, refraction_index=0 + 10j, axis = 0, angle = 0)

t0.u = u0.u

In [ ]:
WPM_inplace(x0, y0, z0, t0, wavelength)

t0.draw_XZ(y0=0, draw_borders=True, scale='scaled')
t0.draw_XY(z0=3)